In [2]:
%load_ext twisted_ipython
%autoawait twisted

The twisted_ipython extension is already loaded. To reload it, use:
  %reload_ext twisted_ipython


In [45]:
import attr
from twisted.internet.defer import Deferred

from korbenware.dbus import dbus_attr, Str

In [46]:
@attr.s
class Request:
    message = dbus_attr(field=Str)
    
@attr.s
class Response:
    message = dbus_attr(field=Str)

In [47]:
class Underlying:
    def echo(self, message):
        print(f'message: {message}')

In [54]:
class Context:
    def __init__(self):
  
    def send(self, argv):
        pass
        
    def call(self, method, *arg, **kwarg):
        pass
        
    async def receive(self, ret):
        self.receiving.callback(ret)
        return await self.received
      
      
class ClientPacker(Context):
  


class Runner:
    def __init__(self, fn):
        self.fn = fn
  
    async def client_side_pack(self, raw_input):
        ctx = Context()

        session = self.fn(ctx, raw_input)
        
        assembled_payload = await ctx.sending
        return assembled_payload    
      
    async def server_side_unpack(self, packed_input):
        ctx = Context()
        
        # Stub packing a message
        session = self.fn(ctx, 'stub input')
        _ = await ctx.sending
        
        ctx.sent(packed_input)  # bon voyage
        
    async def server_side_pack(self, raw_output):
        ctx = Context()
        
        # Stub packing *and* receiving/calling a message
        session = self.fn(ctx, 'stub input')
        stub_packed = await ctx.sending
        ctx.sent(stub_packed)
        stub_ret = await ctx.call('stub method', stub_packed)
        
        ctx.called(raw_output)  # pack and send!
      
    async def client_side_unpack(self, packed_output):
        ctx = Context()
        
        session = self.fn(ctx, 'stub input')
        
        print('waiting for sending')
        stub_packed = await ctx.sending
        
        print('sending sent')
        ctx.sent(stub_packed)
        
        print('waiting for called')
        stub_ret = await ctx.call('stub method', stub_packed)
        
        print('calling is called')
        ctx.called(stub_ret)
        rv = await ctx.receiving
        
        print('sending')
        await ctx.receive(rv)
        ctx.received(rv.message)
        
    async def remote(self, request):
        return Response(message='stubbed remote response')
      
    async def __call__(self, *arg, **kwarg):
        return await self.client_side_unpack(
            await self.remote(await self.client_side_pack(*arg))
        )
        


def dbus_method(argv_sig, ret_sig):
    # So we would know a'priori what the return values would
    # look like
    #
    # Due to dbus not having nullable types, these will also
    # likely generate "sane" default values
    
    # We can control how the context gets constructed
    # TODO: 
    def decorator(f):
        return Runner(f)
    return decorator


@dbus_method([Request], Response)
async def mad_science(ctx, message):
  
    print('creating underlying and such')
    underlying = Underlying
    req = Request(message=message)
    
    print('awaiting send')
    payload = await ctx.send(request)

    print('calling')
    rv = await ctx.call('echo', payload[0].message)
    response = Response(message=rv)
    
    print('receiving')
    rv = await ctx.receive(response)
                        
    return rv.message
    

In [55]:
await mad_science('hello world')


TimeoutError: 

CancelledError: 